In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from utils.poetry import *
from utils.base import *
from utils.transformer_tools import *

from tqdm import tqdm

import youtokentome as yttm
import mauve

C:\Users\Ранагон\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
horus = load_chunks("./datasets/DarkTower/DT14_red.txt", chunk_size=300)

ALL_LEN = len(horus)
len_for_steps = int(ALL_LEN / 3)
horus_for_GAN = horus[len_for_steps * 2:]

for_mauve_test = horus_for_GAN[len(horus_for_GAN)-100:]

print(len(for_mauve_test))

100


In [3]:
BPE_MODEL_FILENAME = './models/tokens/dt_bpe_2000.yttm'
tokenizer = yttm.BPE(BPE_MODEL_FILENAME)

In [4]:
G_model = Language_Model(
    vocab_size = tokenizer.vocab_size(),
    embedding_size = 512,
    backbone = Transformer_Encoder(
        nn.TransformerEncoderLayer(
            d_model = 512,
            nhead = 16,
            dim_feedforward = 1024,
            dropout = 0.3
        ),
        num_layers=5
    ),
    emb_dropout = 0.2
)

G_model_gan = Language_Model(
    vocab_size = tokenizer.vocab_size(),
    embedding_size = 512,
    backbone = Transformer_Encoder(
        nn.TransformerEncoderLayer(
            d_model = 512,
            nhead = 16,
            dim_feedforward = 1024,
            dropout = 0.3
        ),
        num_layers=5
    ),
    emb_dropout = 0.2
)

In [5]:
G_model_small = Language_Model(
    vocab_size = tokenizer.vocab_size(),
    embedding_size = 512,
    backbone = Transformer_Encoder(
        nn.TransformerEncoderLayer(
            d_model = 512,
            nhead = 16,
            dim_feedforward = 1024,
            dropout = 0.3
        ),
        num_layers=5
    ),
    emb_dropout = 0.2
)

G_model_small_gan = Language_Model(
    vocab_size = tokenizer.vocab_size(),
    embedding_size = 512,
    backbone = Transformer_Encoder(
        nn.TransformerEncoderLayer(
            d_model = 512,
            nhead = 16,
            dim_feedforward = 1024,
            dropout = 0.3
        ),
        num_layers=5
    ),
    emb_dropout = 0.2
)

In [6]:
G_model_gan.load_state_dict(torch.load('./models/GAN/DT/1/model_G.pth'))
G_model_gan.cuda()

G_model.load_state_dict(torch.load('./models/Lord/DT/1/Lord.pth'))
G_model.cuda()

G_model_small_gan.load_state_dict(torch.load('./models/GAN/DT/2/model_G.pth'))
G_model_gan.cuda()

G_model_small.load_state_dict(torch.load('./models/Lord/DT/2/Lord.pth'))
G_model.cuda()

device = "cuda"

In [7]:
loop_generator = Gen_method(G_model, tokenizer)
fake_texts = []

loop_generator_gan = Gen_method(G_model_gan, tokenizer)
fake_texts_gan = []

loop_generator_small = Gen_method(G_model_small, tokenizer)
fake_texts_small = []

loop_generator_small_gan = Gen_method(G_model_small_gan, tokenizer)
fake_texts_small_gan = []

In [8]:
for i in tqdm(range(len(for_mauve_test))):
    fake_texts.append(loop_generator(for_mauve_test[i], N_cut=15, need_reweight=True, temper=0.3, alpha=0.15)[0])
    fake_texts_gan.append(loop_generator_gan(for_mauve_test[i], N_cut=15, need_reweight=True, temper=0.3, alpha=0.15)[0])
    fake_texts_small.append(loop_generator_small(for_mauve_test[i], N_cut=15, need_reweight=True, temper=0.3, alpha=0.15)[0])
    fake_texts_small_gan.append(loop_generator_small_gan(for_mauve_test[i], N_cut=15, need_reweight=True, temper=0.3, alpha=0.15)[0])

  0%|          | 0/100 [00:00<?, ?it/s]d:\Programs\DigitalFaculty\utils\poetry.py:205: RuntimeWarning: invalid value encountered in log
  distribution = np.log(original) / temperature
d:\Programs\DigitalFaculty\utils\poetry.py:267: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  next_tokens_logproba = F.log_softmax(next_tokens_logits)
100%|██████████| 100/100 [07:15<00:00,  4.35s/it]


In [9]:
print(fake_texts[4])
print(fake_texts_gan[4])
print(fake_texts_small[4])
print(fake_texts_small_gan[4])

to know you as it is.” “I don’t know, but that’s a name what it really?” Susannah asked. “Yes.” “Then we’ll be drumoned. “Is it possible, if you want to use
to know you as soon begin,” said Mr. Bissette’s name complit. “I A Boy YORK.” The word came out with my mindfirection for a couple of
to know you as your Final Esay, and Engineer Bob and Engineer Bob heard Engineer Bob inscome, Charlie was Charlie’s. Charlie the Ch
to know you as me?” “I don’t know.” “I don’t know,” Roland said, “but that was apt to dollar it.” “If we’re going to kill use the Beam.” “I’m sure


In [10]:
save_texts_to_file(fake_texts_gan, "./datasets/DarkTower/DT_fake_rankgen.txt")
save_texts_to_file(fake_texts, "./datasets/DarkTower/DT_fake_rankgen_old.txt")
save_texts_to_file(fake_texts_small_gan, "./datasets/DarkTower/DTS_fake_rankgen.txt")
save_texts_to_file(fake_texts_small, "./datasets/DarkTower/DTS_fake_rankgen_old.txt")

In [11]:
out_old = mauve.compute_mauve(p_text=for_mauve_test, q_text=fake_texts)
print(out_old.mauve)

out_gan = mauve.compute_mauve(p_text=for_mauve_test, q_text=fake_texts_gan)
print(out_gan.mauve)

out_old_small = mauve.compute_mauve(p_text=for_mauve_test, q_text=fake_texts_small)
print(out_old_small.mauve)

out_gan_small = mauve.compute_mauve(p_text=for_mauve_test, q_text=fake_texts_small_gan)
print(out_gan_small.mauve)

Featurizing q: 100%|██████████| 100/100 [01:01<00:00,  1.62it/s]


0.46440026495076625


Featurizing q: 100%|██████████| 100/100 [01:02<00:00,  1.60it/s]


0.38434915683708215


Featurizing q: 100%|██████████| 100/100 [01:03<00:00,  1.58it/s]


0.1635476586189541


Featurizing q: 100%|██████████| 100/100 [01:04<00:00,  1.55it/s]


0.311050106387855
